# Year-on-Year Species Trend Chart

This notebook generates and exports a year-on-year tredn of sightings for a species, optionally limited to a single location. To use it, update the year range, location, species and required export format in the first code cell, below, before running the notebook.

In [ ]:
# Years to report on
start_year = ""
end_year = ""

# Optional location name to report on. If left blank, report on all locations
location = ""

# Species to report on
species = ""

# Export format for the trend chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "species_year_on_year.sql"
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Replace the location and year placeholders
query = query.replace("$START_YEAR", start_year) \
             .replace("$END_YEAR", end_year) \
             .replace("$LOCATION", location) \
             .replace("$SPECIES", species)

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case='upper'))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["NATURE_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found for species '{species}' from '{start_year}' to '{end_year}"
    if location:
        message += f" at '{location}"
    raise ValueError(message)


In [ ]:
import re

# Specifically add a separate "year" column to the data frame and aggregate the data
df["Year"] = df["Date"].dt.year
yearly_species_counts = df.groupby(["Year", "Species"])["Count"].sum().reset_index()

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Export the trend data to Excel
clean_species = re.sub("[^0-9a-zA-Z ]+", "", species).replace(" ", "-")
if location:
    clean_location = re.sub("[^0-9a-zA-Z ]+", "", location).replace(" ", "-")
    export_file_name = f"{start_year}-{end_year}-{clean_species}-{clean_location}"
else:
    export_file_name = f"{start_year}-{end_year}-{clean_species}"

export_file_path = export_folder_path / f"{export_file_name}-Trend.xlsx"
yearly_species_counts.to_excel(export_file_path.absolute(), sheet_name="Year On Year Trends")

# Print the data
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(yearly_species_counts)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import linregress

x = yearly_species_counts["Year"]
y = yearly_species_counts["Count"]

# Fit linear regression
slope, intercept, *_ = linregress(x, y)
trend_y = intercept + slope * x

# Plot
plt.figure(figsize=(10, 6))

# Bar chart for actual data
plt.bar(x, y, label='Sightings per Year', color='skyblue')

# Trend line over bars
plt.plot(x, trend_y, color='red', linewidth=2, label='Trend Line')

# Set the title and axis labels and style the chart
title = f"Year On Year Trends for {species}"
if location:
    title += f" at {location}"

plt.title(title)
plt.xlabel("Year")
plt.ylabel("Count")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()